<div >
<img src = "../banner.png" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/DSEA/blob/main/Lecture17/L17_Text_as_Data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Texto como Datos

## Regresión con Texto: Gentzkow and Shapiro

Vamos a usar los datos del estudio de Gentzkow y Shapiro [2010], que analiza cómo los periódicos ajustan su contenido según la orientación política de sus lectores.


<div >
<img src = "figures/gentzgow_shapiro.png" />
</div>

Cargamos los paquetes

In [ ]:
install.packages("pacman")

require("pacman")
p_load("tidyverse","textir","wordcloud")

El conjunto de datos resume el primer año del 109º Congreso de los Estados Unidos (2005) e incluye todos los discursos de ese año de los miembros de la Cámara de Representantes y del Senado.

El texto ya está tokenizado en *bigrams* , después de eliminar las *stopwords* y aplicar un algoritmo de lematización (usando el *Porter stemmer*).

In [ ]:
data("congress109", package = "textir")

La matriz `congress109Counts` contiene el número de veces que cada una de las 1000 frases más comunes fue utilizada en el 109º Congreso por cada uno de los 529 miembros del Congreso (Cámara y Senado). 

Es decir, cada documento corresponde a la transcripción combinada de todos los discursos de un solo orador.

Podemos inspeccionar los conteos de algunas frases específicas para dos oradores:

In [ ]:
congress109Counts[c("Barack Obama","John Boehner"),995:999]

Tambien tenemos información sobre  el partido que pertenecen el estado, la camara, y la proporcion de votos que obtuvo Bush en la eleccion presindecial en el distrito/estado de cada representante

In [ ]:
congress109Ideology[c("Barack Obama","John Boehner"),1:5]

Vamos a tratar de predecir estos votos  a partir de como los congresistas hablan. La idea es que el lenguage apela a los votantes: 

In [ ]:
as.matrix(congress109Counts[c("Tom Price","William Jefferson"),c("death.tax","estate.tax")])

In [ ]:
repshare <- congress109Ideology$repshare
X1 <- as.matrix(congress109Counts)
words<-colnames(X1)

covariance<-apply(X1,2,function(x) cor(x,repshare))

                  

wordcloud(words = words,
          freq = covariance,
          min.freq = 0,
          scale = c(1.5, 0.1), 
          max.words=200, 
          random.order=FALSE, 
          colors=brewer.pal(8, "YlOrRd"))

### Regresión 

$$
RepShare= X\beta  + u
$$

In [ ]:
# Predictores
X <- as(congress109Counts, "dMatrix")

# Outcome
repshare <- congress109Ideology$repshare

In [ ]:
# lasso
lassoslant <- cv.gamlr(X>0, repshare)  

B <- coef(lassoslant$gamlr)[-1,] 

In [ ]:
tail(sort(round(B[B!=0],4)),10)

In [ ]:
head(sort(round(B[B!=0],4)),10)